In [1]:
import numpy as np
import pandas as pd
from policyengine_us import Microsimulation
from policyengine_core.reforms import Reform

/Users/pavelmakarchuk/anaconda3/envs/pe/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:

reform = Reform.from_dict({
  "gov.contrib.repeal_state_dependent_exemptions.in_effect": {
    "2024-01-01.2100-12-31": True
  }
}, country_id="us")


baseline = Microsimulation(dataset='pooled_3_year_cps_2023')
reformed = Microsimulation(reform=reform, dataset='pooled_3_year_cps_2023')
baseline_income = baseline.calculate("household_net_income", period=2024)
reformed_income = reformed.calculate("household_net_income", period=2024)
difference_income = reformed_income - baseline_income

In [3]:
difference_income.sum()

-5546086499.324212

In [4]:
# Get state codes
state_code = baseline.calculate("state_code", period=2024)

# Get household weights for accurate aggregation
weights = baseline.calculate("household_weight", period=2024)

# Create a DataFrame with the results
results_df = pd.DataFrame({
    'state_code': state_code,
    'income_change': difference_income,
    'weight': weights
})

# Calculate state-level statistics
state_impacts = (
    results_df
    .groupby('state_code')
    .agg({
        'income_change': lambda x: np.average(x, weights=results_df.loc[x.index, 'weight']),
        'weight': 'sum'
    })
    .reset_index()
)

In [5]:
# Calculate total income impact
total_impact_by_state = (
    results_df
    .groupby('state_code')
    .apply(lambda x: np.sum(x['income_change'] * x['weight']))
    .reset_index(name='total_income_impact')
)

# Merge total impact into main results
state_impacts = state_impacts.merge(total_impact_by_state, on='state_code')

# Add households affected
state_impacts['households_affected'] = (
    results_df[results_df['income_change'] != 0]
    .groupby('state_code')['weight']
    .sum()
    .reindex(state_impacts['state_code'])
    .fillna(0)
)

# Calculate percentage of households affected
state_impacts['percent_households_affected'] = (
    state_impacts['households_affected'] / state_impacts['weight'] * 100
)

# Format the results
state_impacts['avg_income_change'] = state_impacts['income_change'].round(2)
state_impacts['total_income_impact'] = (state_impacts['total_income_impact'] / 1_000_000).round(3)
state_impacts['households_affected'] = state_impacts['households_affected'].round(0)
state_impacts['percent_households_affected'] = state_impacts['percent_households_affected'].round(2)
state_impacts['weight'] = state_impacts['weight'].round(0)

# Sort by total impact
state_impacts_sorted = state_impacts.sort_values('total_income_impact')

# Rename columns for clarity
state_impacts_sorted = state_impacts_sorted.rename(columns={
    'weight': 'total_households',
    'avg_income_change': 'avg_income_change_per_household',
    'total_income_impact': 'total_state_income_impact'
})

# Calculate national totals
national_totals = pd.DataFrame({
    'state_code': ['TOTAL'],
    'total_households': [state_impacts_sorted['total_households'].sum()],
    'avg_income_change_per_household': [(state_impacts_sorted['total_state_income_impact'].sum() / 
                                       state_impacts_sorted['total_households'].sum())],
    'total_state_income_impact': [state_impacts_sorted['total_state_income_impact'].sum()]
})

# Combine state results with national totals
final_results = pd.concat([state_impacts_sorted, national_totals])

# Display results
print("\nState-by-State Impact Analysis:")
print(final_results[[
    'state_code',
    'total_households',
    'avg_income_change_per_household',
    'total_state_income_impact'
]].to_string())


State-by-State Impact Analysis:
   state_code  total_households  avg_income_change_per_household  total_state_income_impact
4          CA        14365824.0                      -161.660004                  -2322.313
22         MI         4228719.0                      -113.089996                   -478.246
10         GA         4348380.0                       -94.570000                   -411.228
23         MN         2344873.0                      -166.809998                   -391.144
14         IL         5187204.0                       -61.880001                   -320.991
34         NY         7896529.0                       -28.510000                   -225.137
40         SC         2221951.0                      -100.739998                   -223.837
31         NJ         3578201.0                       -44.169998                   -158.049
20         MD         2319566.0                       -55.279999                   -128.216
35         OH         4992720.0                

In [6]:
final_results.to_csv('state_impacts_detailed.csv', index=False)

In [9]:
# Define states with exemptions
state_to_exemption = {
    'AL': ['al_personal_exemption'], 'CA': ['ca_exemptions'],
    'GA': ['ga_exemptions'], 'HI': ['hi_exemptions'],
    'IN': ['in_base_exemptions'], 'IL': ['il_dependent_exemption'],
    'KS': ['ks_exemptions'], 'LA': ['la_dependents_exemption'],
    'MD': ['md_total_personal_exemptions'], 'MI': ['mi_exemptions'],
    'MN': ['mn_exemptions'], 'MS': ['ms_dependents_exemption'],
    'MT': ['mt_dependent_exemptions_person'], 'MA': ['ma_income_tax_exemption_threshold'],
    'NC': ['nc_child_deduction'], 'NM': ['nm_deduction_for_certain_dependents'],
    'NE': ['ne_exemptions'], 'NJ': ['nj_dependents_exemption'],
    'NY': ['ny_exemptions'], 'OH': ['oh_personal_exemptions'],
    'OK': ['ok_exemptions'], 'RI': ['ri_exemptions'],
    'SC': ['sc_dependent_exemption'], 'UT': ['ut_exemptions'],
    'VA': ['va_personal_exemption'],
    'VT': ['vt_personal_exemptions'], 'WI': ['wi_exemptions'],
    'WV': ['wv_personal_exemption']
}

# Get required variables
state_codes = baseline.calculate("state_code", period=2024)
num_children = baseline.calculate("tax_unit_children", map_to='household', period=2024)
baseline_tax = baseline.calculate("state_income_tax_before_refundable_credits", map_to='household', period=2024)
reformed_tax = reformed.calculate("state_income_tax_before_refundable_credits", map_to='household', period=2024)

# Create results DataFrame
results = pd.DataFrame({
    'state_code': state_codes,
    'num_children': num_children,
    'zero_tax_both': (np.abs(baseline_tax) < 0.01) & (np.abs(reformed_tax) < 0.01)
})

# Filter for states we want to analyze (exemption states + TX for verification)
states_to_show = list(state_to_exemption.keys()) + ['TX']
results = results[results['state_code'].isin(states_to_show)]

# Get household weights
weights = baseline.calculate("household_weight", period=2024)

# Add weights to results DataFrame
results['weight'] = weights

# Calculate weighted total children by state
total_children = (
    results
    .groupby('state_code')
    .apply(lambda x: np.sum(x['num_children'] * x['weight']))
    .reset_index(name='total_children')
)

# Calculate weighted children with zero tax in both scenarios
zero_tax_children = (
    results[results['zero_tax_both']]
    .groupby('state_code')
    .apply(lambda x: np.sum(x['num_children'] * x['weight']))
    .reset_index(name='children_zero_tax')
)

# Rest of the code remains the same
state_results = total_children.merge(zero_tax_children, on='state_code', how='left')
state_results['percent_zero_tax'] = (
    state_results['children_zero_tax'] / state_results['total_children'] * 100
).round(1)

state_results = state_results.sort_values('percent_zero_tax', ascending=False)
print("\nChildren in households with zero tax in both scenarios (weighted):")
print(state_results.to_string(float_format=lambda x: f"{x:,.1f}"))




Children in households with zero tax in both scenarios (weighted):
   state_code  total_children  children_zero_tax  percent_zero_tax
23         TX     7,005,303.5        7,005,303.5             100.0
22         SC     1,179,198.5          380,843.5              32.3
19         OH     2,536,852.3          665,523.0              26.2
17         NM       460,692.2          118,240.1              25.7
2          GA     2,362,779.7          471,650.4              20.0
6          KS       724,014.4          133,655.5              18.5
28         WV       378,390.8           68,815.4              18.2
24         UT       943,210.6          172,035.9              18.2
12         MS       628,728.5          114,058.6              18.1
1          CA     7,696,760.0        1,359,631.5              17.7
20         OK       959,907.1          152,208.6              15.9
9          MD     1,287,321.9          192,272.0              14.9
7          LA     1,041,173.6          114,304.2             